In [2]:
import numpy as np

In [3]:
data = open('../data/names.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print ('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

data has 228145 characters, 27 unique.


In [4]:
print(ix_to_char)

{0: 'o', 1: 'h', 2: 'u', 3: 'p', 4: 'a', 5: '\n', 6: 's', 7: 'q', 8: 'w', 9: 'z', 10: 'n', 11: 'b', 12: 'y', 13: 'g', 14: 'j', 15: 'm', 16: 'c', 17: 'f', 18: 'l', 19: 'k', 20: 'd', 21: 'i', 22: 't', 23: 'v', 24: 'e', 25: 'r', 26: 'x'}


In [5]:
def lossFun(inputs, targets, hprev):
    """
    inputs, targets are both list of integers.
    hprev is Hx1 array of initial hidden state
    returns the loss, gradients on model parameters, and last hidden state
    """
    # Create four dictionaries
    xs, hs, ys, ps = {}, {}, {}, {}
    # Copy the last hidden state into hprev
    hs[-1] = np.copy(hprev)
    loss = 0
    
    ### Forward pass (each time step)
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
        # At this time step, input an encoded char, as a (27, 5) one hot vector
        xs[t][inputs[t]] = 1
        # At this time step, follow the formulas to get the hidden state at this time step
        # (100,27) @ (27, 1) = (100, 1), (100, 100) @ (100, 1) = (100, 1) + (100, 1) -> (100, 1) (column vector of 100 outputs
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state

        # At this time step, compute the output state
        # (27, 100) @ (100, 1) = (27, 1) + (27, 1) = (1, 27)
        ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars

        # Normalize our probabiltiies using softmax
        # ps = (1, 27)
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars

        # Take the loss of the correct output probabilities
        loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
    
    ### Backward pass: compute gradients going backwards
    # dWxh (100, 27), dWhh (100, 100), dWhy = (27, 100)
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    # dbh (100, 1),  dby (27, 1)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    # (100, 1)
    # Start off with zero
    dhnext = np.zeros_like(hs[0]) 
    
    for t in reversed(range(len(inputs))):
        # Backpropogate through the softmax to the logits
        # (27, 5)
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here

        # (27, 5) @ (5, 100) = (27, 100)
        dWhy += np.dot(dy, hs[t].T)
        # (27, 5) -> This secrety should sum across the columns dim = 1 to produce (27, 1)
        dby += dy

        # dh(t), dh(t)raw
        # Backpropogate into the previous layer (100, 27) @ (27, 5) = (100, 5)
        dh = np.dot(Why.T, dy) + dhnext
        # Backprop through tanh nonlinearity (element-wise forward -> element-wise backward)
        dhraw = (1 - hs[t] * hs[t]) * dh 

        # dWxh, dWhh, dbh
        # dWxh = (100, 5) @ (5, 27) = (100, 27)
        dWxh += np.dot(dhraw, xs[t].T)
        # dWhh = (100, 5) (5, 100) = (100, 100)
        dWhh += np.dot(dhraw, hs[t-1].T)
        # Calculate dbh and dWxh
        dbh += dhraw

        # Derive part of the next time step's gradient
        dhnext = np.dot(Whh.T, dhraw)

    # Clipping gradients
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam)
    
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]


In [6]:
def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes


In [9]:
# smooth_loss = -np.log(1.0/vocab_size)*seq_length
# smooth_loss

In [11]:
# if p+seq_length+1 >= len(data) or n == 0: 
#     hprev = np.zeros((hidden_size,1)) # reset RNN memory
#     p = 0 # go from start of data
# inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
# targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

NameError: name 'p' is not defined

In [72]:
# print([ix_to_char[ch] for ch in inputs])

['m', 'm', 'a', '\n', 'o', 'l', 'i', 'v', 'i', 'a', '\n', 'a', 'v', 'a', '\n', 'i', 's', 'a', 'b', 'e', 'l', 'l', 'a', '\n', 's']


In [71]:
 # inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]

In [70]:
# p = 1

In [14]:
# Hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

# Model parameters
Wxh = np.random.randn(hidden_size, vocab_size) * 0.01 # input to hidden - (100, 27)
Whh = np.random.randn(hidden_size, hidden_size) * 0.01 # hidden to hidden (100, 100)
Why = np.random.randn(vocab_size, hidden_size) * 0.01 # hidden to output (27, 100)
bh = np.zeros((hidden_size, 1)) # hidden bias (100, 1)
by = np.zeros((vocab_size, 1)) # output bias (27, 1)

n, p = 0, 0

mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)

mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad

smooth_loss = -np.log(1.0/vocab_size) * seq_length # loss at iteration 0

while True:
    # Prepare inputs (Sweeping from left to right in steps seq_length long)
    if p+seq_length+1 >= len(data) or n == 0: 
        hprev = np.zeros((hidden_size,1)) # reset RNN memory
        p = 0 # Go from start of data
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    print(len(inputs))
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

    # Sample from the model now and then, and print the result
    if n % 100 == 0:
        sample_ix = sample(hprev, inputs[0], 200)
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print ('----\n %s \n----' % (txt, ))

    # Forward seq_length characters through the net and fetch gradient
    
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n % 100 == 0: 
        print ('iter %d, loss: %f' % (n, smooth_loss)) # print progress
  
    # perform parameter update with Adagrad
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update
        
    p += seq_length # move data pointer
    n += 1 # iteration counter 

25
----
 oplltkmmrxrtfqrkxxbdrtyivexhjpprelrltzt
iouxjqiyweoldf

hqymarmzibtsfzkitrtlzyrvygzhybvnvkppekrshikjjnnaghnsildgbfvgykzfnneydjwsfbxnzlbmlrzpktaafdwa
fuio
aeatodgojabvwojssxupp
bxkxfrvlllwehrfbukzmkvjx 
----
iter 0, loss: 82.395922
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
----
 
lvailyzeadsraoo
evvhfezymvhirpgryelrifjbaeexai

magvseeloy
hspuiaaam
zeweaayszyeeair
oblailedmidamuar
dliihrnbearaatarov
elyrloeeiv
rpyeaaan
tpaeafqexod

rrvihpi
varf
s
nrrnhijdbhgxo
clk
xrulajmv
wio 
----
iter 100, loss: 83.038869
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25

KeyboardInterrupt: 